In [ ]:
import pymc as pm
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import arviz as az
import pandas as pd
import arviz as az

In [ ]:
iris = pd.read_csv("iris.csv")

df = iris.query("species == ('setosa', 'versicolor')")
y_0 = pd.Categorical(df['species']).codes
x_n = 'sepal_length'
x_0 = df[x_n].values
x_c = x_0 - x_0.mean()

In [ ]:
with pm.Model() as model_0:
  α = pm.Normal('α', mu=0, sigma=10)
  β = pm.Normal('β', mu=0, sigma=10)
  μ = α + pm.math.dot(x_c, β)
  θ = pm.Deterministic('θ', pm.math.sigmoid(μ))
  bd = pm.Deterministic('bd', -α/β)
  yl = pm.Bernoulli('yl', p=θ, observed=y_0)
  idata_0 = pm.sample(1000, return_inferencedata=True)

In [ ]:
posterior_0 = idata_0.posterior.stack(samples=("chain", "draw"))
theta = posterior_0['θ'].mean("samples")
idx = np.argsort(x_c)
plt.plot(x_c[idx], theta[idx], color='C2', lw=3)
plt.vlines(posterior_0['bd'].mean(), 0, 1, color='k')
bd_hpd = az.hdi(posterior_0['bd'].values)
plt.fill_betweenx([0, 1], bd_hpd[0], bd_hpd[1], color='k', alpha=0.5)
plt.scatter(x_c, np.random.normal(y_0, 0.02),
marker='.', color=[f'C{x}' for x in y_0])
az.plot_hdi(x_c, posterior_0['θ'].T, color='C2', smooth=False)
plt.xlabel(x_n)
plt.ylabel('θ', rotation=0)
# use original scale for xticks
locs, _ = plt.xticks()
plt.xticks(locs, np.round(locs + x_0.mean(), 1))